# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning
# Feature Selection

**Grupo 30: Datatouille**

**http://fdelmazo.github.io/7506-Datos/**

**Siendo este un proceso muy costoso en tiempo, se corre una vez y se guardan los resultados para luego poder importarlos**

In [2]:
def get_feature_selection():
    # Se esconde en un def para poder importarlo

    return {

        'best_features_progresivo': ['timestamp_last_checkout',
                                     'doy_last_checkout',
                                     'woy_last_checkout',
                                     'percentage_last_month_checkouts',
                                     'has_checkout_month_5',
                                     'total_checkouts_month_5',
                                     'days_between_last_event_and_checkout',
                                     'kmeans_5',
                                     'total_events_ad_session',
                                     'dom_last_checkout',
                                     'total_events',
                                     'days_to_last_checkout',
                                     'kmeans_3',
                                     'total_events_month_5',
                                     'total_events_ad_session_month_5',
                                     'total_viewed_products',
                                     'dow_last_checkout',
                                     'total_sessions',
                                     'total_viewed_products_month_5',
                                     'timestamp_last_event',
                                     'total_events_lw',
                                     'total_events_ad_session_lw',
                                     'total_checkouts',
                                     'total_max_viewed_product',
                                     'total_ad_sessions',
                                     'percentage_last_week_checkouts',
                                     'percentage_last_month_activity',
                                     'total_last_week_max_viewed_brand',
                                     'days_to_last_viewed_product',
                                     'doy_last_event',
                                     'days_to_last_event',
                                     'avg_events_per_ad_session',
                                     'dom_last_event',
                                     'total_sessions_month_5',
                                     'avg_events_per_session',
                                     'kmeans_6',
                                     'total_viewed_products_months_1_to_4',
                                     'total_viewed_products_lw',
                                     'total_sessions_last_week',
                                     'var_viewed',
                                     'total_ad_sessions_month_5',
                                     'woy_last_event',
                                     'days_to_last_conversion',
                                     'total_brand_listings',
                                     'percentage_max_viewed_product',
                                     'total_events_months_1_to_4',
                                     'total_brand_listings_lw',
                                     'total_sessions_lw',
                                     'total_last_week_max_viewed_model',
                                     'has_checkout_lw',
                                     'total_checkouts_lw',
                                     'total_sessions_months_1_to_4',
                                     'percentage_last_week_viewed_products',
                                     'total_session_checkout',
                                     'percentage_regular_celphones_activity',
                                     'percentage_last_month_viewed_products',
                                     'percentage_last_week_activity',
                                     'percentage_last_week_brand_listings',
                                     'percentage_last_week_max_viewed_brand',
                                     'percentage_last_week_max_viewed_model',
                                     'dow_last_event',
                                     'total_ad_sessions_lw',
                                     'percentage_session_ad',
                                     'total_sessions_month_1',
                                     'total_conversions_month_2',
                                     'total_session_checkouts_month_1',
                                     'total_events_month_1',
                                     'doy_last_conversion',
                                     'total_checkouts_month_2',
                                     'total_ad_sessions_month_1',
                                     'total_session_conversions_month_1',
                                     'total_events_ad_session_month_1',
                                     'total_viewed_products_month_2',
                                     'dom_last_conversion',
                                     'has_checkout_month_1',
                                     'has_conversion_month_1',
                                     'dow_last_conversion',
                                     'dom_last_viewed_product',
                                     'woy_last_conversion',
                                     'conversion_gt_media',
                                     'total_conversions',
                                     'total_session_conversion',
                                     'cant_visitas_faq_ecommerce',
                                     'cant_visitas_customer_service',
                                     'has_event_last_week',
                                     'ratio_sessions_last_week_over_total',
                                     'percentage_session_conversion',
                                     'cant_viewed_brand_last_conversion',
                                     'has_checkout',
                                     'has_conversion',
                                     'doy_last_viewed_product',
                                     'total_viewed_products_month_1',
                                     'total_checkouts_month_1',
                                     'percentage_last_month_conversions',
                                     'total_conversions_month_1',
                                     'last_conversion_price',
                                     'last_conversion_sku',
                                     'woy_last_viewed_product',
                                     'timestamp_last_conversion',
                                     'dow_last_viewed_product',
                                     'timestamp_last_viewed_product',
                                     'total_events_ad_session_month_3',
                                     'total_events_month_2',
                                     'total_ad_sessions_month_4',
                                     'total_session_conversions_month_5',
                                     'total_session_checkouts_month_5',
                                     'total_events_month_3',
                                     'total_sessions_month_3',
                                     'total_conversions_month_5'],
        
        'best_features_forward': ['doy_last_checkout',
                                 'total_events',
                                 'dom_last_event',
                                 'percentage_session_ad',
                                 'avg_events_per_session',
                                 'total_events_lw'],
  
          'best_features_backward' :  ['total_conversions', 
                                     'total_events', 
                                     'total_session_conversion', 
                                     'total_events_ad_session', 
                                     'total_ad_sessions', 
                                     'avg_events_per_session',
                                     'avg_events_per_ad_session', 
                                     'percentage_session_ad',
                                     'percentage_session_conversion',
                                     'has_checkout', 
                                     'has_conversion',
                                     'total_viewed_products_month_1',
                                     'total_checkouts_month_1', 
                                     'total_conversions_month_1', 
                                     'total_events_month_1', 
                                     'total_sessions_month_1',
                                     'total_session_checkouts_month_1', 
                                     'total_session_conversions_month_1', 
                                     'total_events_ad_session_month_1',
                                     'total_ad_sessions_month_1', 
                                     'has_checkout_month_1',
                                     'has_conversion_month_1', 
                                     'total_viewed_products_month_2',
                                     'total_checkouts_month_2', 
                                     'total_conversions_month_2',
                                     'total_events_month_2', 
                                     'total_sessions_month_2', 
                                     'total_session_checkouts_month_2', 
                                     'total_session_conversions_month_2', 
                                     'total_events_ad_session_month_2', 
                                     'total_ad_sessions_month_2',
                                     'has_checkout_month_2', 
                                     'has_conversion_month_2', 
                                     'total_viewed_products_month_3',
                                     'total_checkouts_month_3',
                                     'total_conversions_month_3',
                                     'total_events_month_3',
                                     'total_sessions_month_3', 
                                     'total_session_checkouts_month_3', 
                                     'total_session_conversions_month_3', 
                                     'total_events_ad_session_month_3', 
                                     'total_ad_sessions_month_3', 
                                     'has_checkout_month_3', 
                                     'has_conversion_month_3',
                                     'total_viewed_products_month_4',
                                     'total_checkouts_month_4', 
                                     'total_conversions_month_4',
                                     'total_session_checkouts_month_4',
                                     'total_session_conversions_month_4',
                                     'total_events_ad_session_month_4',
                                     'total_ad_sessions_month_4', 
                                     'has_checkout_month_4', 
                                     'has_conversion_month_4',
                                     'total_viewed_products_month_5',
                                     'total_events_month_5', 
                                     'total_sessions_month_5', 
                                     'total_session_checkouts_month_5',
                                     'total_session_conversions_month_5', 
                                     'total_events_ad_session_month_5',
                                     'total_ad_sessions_month_5', 
                                     'total_checkouts_months_1_to_4', 
                                     'total_conversions_months_1_to_4',
                                     'total_session_conversions_months_1_to_4', 
                                     'total_events_ad_session_months_1_to_4',
                                     'total_ad_sessions_months_1_to_4', 
                                     'has_conversion_months_1_to_4', 
                                     'amount_of_months_that_has_bought',
                                     'timestamp_last_event',
                                     'days_to_last_event',
                                     'days_to_last_conversion', 
                                     'days_to_last_viewed_product',
                                     'doy_last_event',
                                     'dow_last_event', 
                                     'dom_last_event',
                                     'woy_last_event',
                                     'doy_last_checkout', 
                                     'woy_last_checkout',
                                     'dow_last_conversion', 
                                     'dom_last_conversion', 
                                     'woy_last_conversion',
                                     'dow_last_viewed_product',
                                     'dom_last_viewed_product', 
                                     'woy_last_viewed_product',
                                     'last_conversion_price',
                                     'percentage_last_week_activity',
                                     'percentage_last_week_conversions', 
                                     'percentage_last_week_viewed_products', 
                                     'percentage_last_month_activity',
                                     'percentage_last_month_checkouts', 
                                     'percentage_regular_celphones_activity',
                                     'var_viewed', 
                                     'conversion_gt_media',
                                     'total_max_viewed_product',
                                     'cant_viewed_brand_last_conversion',
                                     'ratio_sessions_last_week_over_total',
                                     'has_event_last_week', 
                                     'cant_visitas_customer_service', 
                                     'cant_visitas_faq_ecommerce'],

        'best_features_stepwise' : ['doy_last_checkout',
                                     'total_events',
                                     'dom_last_event',
                                     'percentage_session_ad',
                                     'avg_events_per_session',
                                     'total_events_lw']
}

---
---
---

In [7]:
import nbimporter # pip install nbimporter
import pandas as pd
from parameter_tuning import get_hiper_params
import submission_framework as SF

df_users = pd.read_csv('data/user-features.csv',low_memory=False).set_index('person')
df_y = pd.read_csv('data/labels_training_set.csv').groupby('person').sum()

seed=42
hiper_params = get_hiper_params()

df_users.head()

,total_brand_listings,total_viewed_products,total_checkouts,total_conversions,total_events,total_sessions,total_session_checkout,total_session_conversion,total_events_ad_session,total_ad_sessions,...,percentage_l2w_week_activity,percentage_l2w_brand_listings,percentage_l2w_viewed_products,percentage_l2w_checkouts,percentage_l2w_conversions,kmeans_3,kmeans_5,kmeans_6,kmeans_15,kmeans_25
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,0.0,0.0,3.0,0.0,6,3.0,3.0,0.0,0.0,0.0,...,1.000000,0.000000,0.000000,0.500000,0.000000,1,1,4,0,14
00091926,25.0,372.0,2.0,0.0,448,34.0,2.0,0.0,54.0,9.0,...,0.582589,0.006696,0.511161,0.004464,0.000000,1,1,1,3,7
00091a7a,5.0,3.0,0.0,0.0,10,1.0,0.0,0.0,10.0,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0,3,3,8,16
000ba417,24.0,153.0,6.0,1.0,206,5.0,4.0,1.0,0.0,0.0,...,1.000000,0.116505,0.742718,0.029126,0.004854,2,2,0,12,0
000c79fe,0.0,3.0,1.0,0.0,17,1.0,1.0,0.0,17.0,1.0,...,1.000000,0.000000,0.176471,0.058824,0.000000,1,1,1,3,7


---
---
---


Usando Random Forest, el algoritmo más estable de los definidos (XGBoost es poco estable) encontramos que combinación de features es la mas favorable (con la métrica Area Under Curve).

Se usa de parametro `n_estimators` un número grande, ya que a mayor cantidad de árboles, más precisa la importancia de los features

In [8]:
from sklearn.ensemble import RandomForestClassifier

model_name = 'random_forest'
params = hiper_params[model_name]
params['n_estimators'] = 500
model = RandomForestClassifier(**params,random_state=seed)
random_forest = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,random_forest)

Model: random_forest - AUC: 0.8424 - AUCPR:0.1624 - Accuracy: 0.9496 


(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
             max_depth=16.5, max_features=0.2, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=0.1, min_samples_split=0.1,
             min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
             oob_score=False, random_state=42, verbose=0, warm_start=False),
 0.8424451094361751)

In [9]:
full_features = df_users.columns.tolist()
cantidad_features = len(full_features)

tried = {}

## Cumulative Importance

Se parte de una lista de todos los features ordeandos segun importancia, y se genera una lista de listas acumulativa de esto. Es decir de `[a,b,c]` se pasa a `[ [a], [a,b], [a,b,c] ]`

Esto se hace porque se esta buscando el 'codo': Los features que hacen que incremente el AUC.

In [10]:
feature_importances = SF.get_feature_importances(df_users,df_y,random_forest)
display(feature_importances)
features_ordenados = feature_importances.index.tolist()
lista_progresiva_de_cols = [features_ordenados[:i] for i in range(1,len(features_ordenados))]

model, max_auc = SF.full_framework_wrapper(df_users,df_y,random_forest)
best_features_progresivo = features_ordenados
features_con_saltos_progresivo = []

for i, cols in enumerate(lista_progresiva_de_cols):
    print(f'\n\nIteración {i+1} de {len(lista_progresiva_de_cols)}\n\n')
    if tuple(cols) not in tried:
        model, auc = SF.full_framework_wrapper(df_users,df_y,random_forest,columns=cols,verbosity=1)
        tried[tuple(cols)] = auc
    else:
        auc = tried[tuple(cols)]
    if auc > max_auc + 0.0001:
        max_auc = auc
        best_features_progresivo = cols
        features_con_saltos_progresivo.append(cols[-1])

,importance
timestamp_last_checkout,0.150733
doy_last_checkout,0.127105
woy_last_checkout,0.121945
percentage_last_month_checkouts,0.082231
total_session_checkouts_month_5,0.072992
total_checkouts_month_5,0.072743
has_checkout_month_5,0.054839
days_to_last_checkout,0.035729
days_between_last_event_and_checkout,0.029386
total_events,0.025736


Model: random_forest - AUC: 0.8424 - AUCPR:0.1624 - Accuracy: 0.9496 


Iteración 1 de 187


Model: random_forest - AUC: 0.8064 - AUCPR:0.1563 - Accuracy: 0.9496 
['timestamp_last_checkout']


Iteración 2 de 187


Model: random_forest - AUC: 0.8140 - AUCPR:0.1578 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout']


Iteración 3 de 187


Model: random_forest - AUC: 0.8171 - AUCPR:0.1584 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout']


Iteración 4 de 187


Model: random_forest - AUC: 0.8353 - AUCPR:0.1662 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'percentage_last_month_checkouts']


Iteración 5 de 187


Model: random_forest - AUC: 0.8363 - AUCPR:0.1664 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'percentage_last_month_checkouts', 'total_session_checkouts_month_5']


Iteración 6 de 187


Model: random_forest - AUC: 0.8444 - AUCPR:0.1687 

Model: random_forest - AUC: 0.8337 - AUCPR:0.1509 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'percentage_last_month_checkouts', 'total_session_checkouts_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'days_to_last_checkout', 'days_between_last_event_and_checkout', 'total_events', 'has_checkout_l2w', 'total_events_month_5', 'total_viewed_products', 'dom_last_checkout', 'total_events_l2w', 'total_checkouts_l2w', 'percentage_l2w_checkouts', 'kmeans_3', 'total_events_lw', 'total_viewed_products_l2w', 'dow_last_checkout', 'total_viewed_products_month_5', 'total_sessions_lw']


Iteración 24 de 187


Model: random_forest - AUC: 0.8343 - AUCPR:0.1507 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'percentage_last_month_checkouts', 'total_session_checkouts_month_5', 'total_checkouts_month_5', 'has_checkout_month_5', 'days_to_last_checkout', 'days_between_last_event_and_checkout', 'total_event

In [11]:
best_features_progresivo

['timestamp_last_checkout',
 'doy_last_checkout',
 'woy_last_checkout',
 'percentage_last_month_checkouts',
 'total_session_checkouts_month_5',
 'total_checkouts_month_5',
 'has_checkout_month_5']

##  Forward Selection

Se parte de una lista vacía y se van agregando todos los features uno por uno.

In [12]:
def step_forward(features_usadas, features_sin_usar):
    if not features_sin_usar: 
        return SF.full_framework_wrapper(df_users,df_y,random_forest,columns=features_usadas,verbosity=1)[1], None
    max_auc = 0
    features_a_usar = None
    la_posta = None
    for f in features_sin_usar:
        features_a_probar = features_usadas + [f]
        if tuple(features_a_probar) not in tried: 
            model, auc = SF.full_framework_wrapper(df_users,df_y,random_forest,columns=features_a_probar,verbosity=1)
            tried[tuple(features_a_probar)] = auc
        else: 
            auc = tried[tuple(features_a_probar)]
        if auc > max_auc + 0.0001:
            la_posta = f
            max_auc = auc
            
    return max_auc, la_posta

In [14]:
CORTE = 15 # Si en x iteraciones no modifico su valor, se corta la ejecución. Puede entrar en un maximo local, pero es muy costoso en tiempo, hay que cortarlo

contador = 0
best_features_forward = None
max_auc = 0
features_usadas = []
features_sin_usar = full_features[:]

for i in range(cantidad_features):
    print(f'\n\nIteración {i+1} de {cantidad_features}')
    print(f'Lo mejor al momento: {best_features_forward} (AUC: {max_auc:.4f}) \n\n')
    auc, feature_a_agregar = step_forward(features_usadas, features_sin_usar)
    features_usadas.append(feature_a_agregar)
    features_sin_usar.remove(feature_a_agregar)
    contador += 1
    if auc > max_auc + 0.0001:
        contador = 0
        best_features_forward = features_usadas[:]
        max_auc = auc   
    if contador == CORTE: break



Iteración 1 de 188
Lo mejor al momento: None (AUC: 0.0000) 


Model: random_forest - AUC: 0.6349 - AUCPR:0.0766 - Accuracy: 0.9496 
['total_brand_listings']
Model: random_forest - AUC: 0.6235 - AUCPR:0.0778 - Accuracy: 0.9496 
['total_viewed_products']
Model: random_forest - AUC: 0.6438 - AUCPR:0.0731 - Accuracy: 0.9496 
['total_checkouts']
Model: random_forest - AUC: 0.5000 - AUCPR:0.0504 - Accuracy: 0.9496 
['total_conversions']
Model: random_forest - AUC: 0.6366 - AUCPR:0.0830 - Accuracy: 0.9496 
['total_events']
Model: random_forest - AUC: 0.6449 - AUCPR:0.0799 - Accuracy: 0.9496 
['total_sessions']
Model: random_forest - AUC: 0.6883 - AUCPR:0.0856 - Accuracy: 0.9496 
['total_session_checkout']
Model: random_forest - AUC: 0.5000 - AUCPR:0.0504 - Accuracy: 0.9496 
['total_session_conversion']
Model: random_forest - AUC: 0.5801 - AUCPR:0.0670 - Accuracy: 0.9496 
['total_events_ad_session']
Model: random_forest - AUC: 0.5823 - AUCPR:0.0646 - Accuracy: 0.9496 
['total_ad_sessions']
M

In [15]:
best_features_forward

##  Backward Elimination

Se parte de una lista con todos los features y se van sacando uno por uno, en busqueda de cual hace que incremente un AUC una vez que se lo remueva.

In [16]:
def step_backward(features):
    if len(features) == 1: 
        print('atroden')
        return SF.full_framework_wrapper(df_users,df_y,random_forest,columns=features,verbosity=1)[1], None
    max_auc = 0
    features_a_usar = None
    la_posta = None
    for f in features:
        features_a_probar = features[:]
        features_a_probar.remove(f)
        if tuple(features_a_probar) not in tried:
            model, auc = SF.full_framework_wrapper(df_users,df_y,random_forest,columns=features_a_probar,verbosity=1)
            tried[tuple(features_a_probar)] = auc
        else:
            auc = tried[tuple(features_a_probar)]
        if auc > max_auc + 0.0001:
            la_posta = f
            max_auc = auc
                
    return max_auc, la_posta

In [19]:
CORTE = 5

contador = 0
features = full_features[:]
best_features_backward = features[:]
max_auc = SF.full_framework_wrapper(df_users,df_y,random_forest,columns=best_features_backward)[1]

for i in range(1,cantidad_features):
    print(f'\n\nIteración {i} de {cantidad_features-1}')
    print(f'Lo mejor al momento: {best_features_backward} (AUC: {max_auc:.4f}) \n\n')
    auc, feature_a_borrar = step_backward(features)
    features.remove(feature_a_borrar)
    contador += 1
    if auc > max_auc + 0.0001:
        contador = 0
        best_features_backward = features[:]
        max_auc = auc
    if contador == CORTE: break

Model: random_forest - AUC: 0.8424 - AUCPR:0.1624 - Accuracy: 0.9496 


Iteración 1 de 187
Lo mejor al momento: ['total_brand_listings', 'total_viewed_products', 'total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkout', 'total_session_conversion', 'total_events_ad_session', 'total_ad_sessions', 'avg_events_per_session', 'avg_events_per_ad_session', 'percentage_session_ad', 'percentage_session_conversion', 'has_checkout', 'has_conversion', 'total_viewed_products_month_1', 'total_checkouts_month_1', 'total_conversions_month_1', 'total_events_month_1', 'total_sessions_month_1', 'total_session_checkouts_month_1', 'total_session_conversions_month_1', 'total_events_ad_session_month_1', 'total_ad_sessions_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_viewed_products_month_2', 'total_checkouts_month_2', 'total_conversions_month_2', 'total_events_month_2', 'total_sessions_month_2', 'total_session_checkouts_month_2', 'total_session_c

In [20]:
best_features_backward

['total_brand_listings',
 'total_viewed_products',
 'total_checkouts',
 'total_conversions',
 'total_events',
 'total_sessions',
 'total_session_checkout',
 'total_session_conversion',
 'total_events_ad_session',
 'total_ad_sessions',
 'avg_events_per_session',
 'avg_events_per_ad_session',
 'percentage_session_ad',
 'percentage_session_conversion',
 'has_checkout',
 'has_conversion',
 'total_viewed_products_month_1',
 'total_checkouts_month_1',
 'total_conversions_month_1',
 'total_events_month_1',
 'total_sessions_month_1',
 'total_session_checkouts_month_1',
 'total_session_conversions_month_1',
 'total_events_ad_session_month_1',
 'total_ad_sessions_month_1',
 'has_checkout_month_1',
 'has_conversion_month_1',
 'total_viewed_products_month_2',
 'total_checkouts_month_2',
 'total_conversions_month_2',
 'total_events_month_2',
 'total_sessions_month_2',
 'total_session_checkouts_month_2',
 'total_session_conversions_month_2',
 'total_events_ad_session_month_2',
 'total_ad_sessions_mo

##  Stepwise Regression

> http://people.stat.sfu.ca/~lockhart/richard/350/08_2/lectures/VariableSelection/web.pdf

Combinación de Forward Selection y Backward Elimination

In [21]:
CORTE = 15
contador = 0
best_features_stepwise = []
max_auc = 0
features_usadas = []
features_sin_usar = full_features[:]

for i in range(cantidad_features):
    print(f'\n\nIteración {i+1} de {cantidad_features}')
    print(f'Lo mejor al momento: {best_features_stepwise} (AUC: {max_auc:.4f}) \n\n')
    auc_loc, feature_a_agregar = step_forward(features_usadas, features_sin_usar)
    features_usadas.append(feature_a_agregar)
    features_sin_usar.remove(feature_a_agregar)
    features_to_stepwisear = features_usadas[:]
    iteraciones = len(features_to_stepwisear)
    
    print(f'\nStepWise: {features_to_stepwisear}\n')
    for i in range(1,iteraciones):
        auc, feature_a_borrar = step_backward(features_to_stepwisear)
        features_to_stepwisear.remove(feature_a_borrar)
        if auc > auc_loc:
            features_usadas = features_to_stepwisear[:]
            auc_loc = auc

    contador += 1
    if auc_loc > max_auc:
        contador = 0
        best_features_stepwise = features_usadas[:]
        max_auc = auc_loc   
        
    if contador == CORTE: break



Iteración 1 de 188
Lo mejor al momento: [] (AUC: 0.0000) 


Model: random_forest - AUC: 0.5584 - AUCPR:0.0570 - Accuracy: 0.9496 
['avg_events_per_ad_session']
Model: random_forest - AUC: 0.6333 - AUCPR:0.0756 - Accuracy: 0.9496 
['percentage_session_ad']
Model: random_forest - AUC: 0.5000 - AUCPR:0.0504 - Accuracy: 0.9496 
['percentage_session_conversion']


KeyboardInterrupt: Proceso muy costoso en tiempo. Se corta la ejecución, que ya sus resultados fueron guardados


In [22]:
best_features_stepwise

[]